In [24]:
from PIL import Image
import pickle, gzip, numpy
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.datasets import mnist
import os

# **Helper Functions** 

In [2]:
def extract_features(dataset, datasetSize):
    featureArray = []
    for i in range(datasetSize):
        imageData = dataset[0][i]
        featureArray.append(imageData)
    return numpy.array(featureArray)

def extract_targets(dataset, datasetSize):
    targetArray = []
    for i in range(datasetSize):
        targetImage = dataset[1][i]
        targetArray.append(targetImage)
    return numpy.array(targetArray)

def one_hot_encoding(dataset,datasetSize):
    oneHotEncodedLabels = numpy.zeros((datasetSize, 10))
    for i in range(datasetSize):
        oneHotEncodedLabels[i][dataset[i]] = 1
    return numpy.array(oneHotEncodedLabels)

def random_initialization_of_array(arr, numOfRows, numOfCols):
    for i in range(numOfRows):
        for j in range(numOfCols):
            arr[i][j] = numpy.random.randint(1,10)/10000;
    return numpy.array(arr)

def softmax_function(num, arr):
    if(numpy.sum(numpy.exp(arr))!=0):
        numerator = numpy.exp(num)
        denominator = numpy.sum(numpy.exp(arr))
        return (numerator/denominator)
    else:
        return 0
    

In [17]:
mnistTrainSize = 50000
mnistValidateSize = 10000
mnistTestSize = 10000
numOfClasses = 10

# Preparing MNIST Feature Dataset

In [18]:
# Loading the training, validation and testing dataset from mnist pickle file
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, validation_set, test_set = pickle.load(f, encoding='latin1')
f.close()

# Processing the dataset for creating numpy array that contains features
mnistFeaturesTrain = extract_features(train_set,mnistTrainSize) #features in training set
mnistFeaturesValidate = extract_features(validation_set,mnistValidateSize) #features in validation set
mnistFeaturesTest = extract_features(test_set,mnistTestSize) #features in testing set

# Processing the dataset for creating numpy array that contains target lables
mnistTargetTrain = extract_targets(train_set,mnistTrainSize) #target labels in training set
mnistTargetValidate = extract_targets(validation_set,mnistValidateSize) #taret labels in validation set
mnistTargetTest = extract_targets(test_set,mnistTestSize) #target labels in testing set

# Training MNIST Dataset using Softmax Logistic Regression

In [19]:
# one-hot encoding of target labels
trainingLabels = one_hot_encoding(mnistTargetTrain, mnistTrainSize)
validationLabels = one_hot_encoding(mnistTargetValidate, mnistValidateSize)
testingLabels = one_hot_encoding(mnistTargetTest, mnistTestSize)

# randomly initializing weight vector
weights = numpy.zeros((784,numOfClasses))
weights = random_initialization_of_array(weights, 784, numOfClasses)

# randomly initializing weight vector
bias = numpy.zeros((1,10))
bias = random_initialization_of_array(bias, 1, numOfClasses)

# weight vector with bias
weights = numpy.concatenate((bias, weights))

# feature vectors with "ones"
trainingFeatures = numpy.insert(mnistFeaturesTrain, 0, 1, axis=1)
validationFeatures = numpy.insert(mnistFeaturesValidate, 0, 1, axis=1)
testingFeatures = numpy.insert(mnistFeaturesTest, 0,1, axis=1)

#### GRADIENT DESCENT SOLUTION

In [20]:
# HyperParameters
learningRate = 0.4
Lambda = 0.5
epochs = 220

In [21]:
# Updating weights for all classes over entire training dataset
for i in range(epochs):
    # creating net input Z
    Z = numpy.dot(trainingFeatures, weights)
    
    #creating the softmax activation vector
    softmax = []
    count = 0
    for row in Z:
        count = count + 1
        data = numpy.zeros((1,10))
        for i in range(numOfClasses):
            data[0][i] = softmax_function(row[i], row)
        softmax.append(data)
    softmax = numpy.array(softmax)
    
    # calculating cost derivative - Cost is Cross Entropy
    costDerivative = numpy.zeros((785,numOfClasses))
    for rowTrain, rowSoftmax, rowLabel in zip(trainingFeatures, softmax, trainingLabels):
        rowTrain = rowTrain.reshape(1,785)
        costDerivative = numpy.add(numpy.dot(numpy.transpose(rowTrain),numpy.subtract(rowLabel,rowSoftmax)), costDerivative)
    costDerivative = costDerivative/mnistTrainSize
    #costDerivative = costDerivative + Lambda*weights
    weights = numpy.subtract(weights, numpy.dot(learningRate, numpy.dot(-1,costDerivative)))

# calculating net input for validation dataset
Zvalidation = numpy.dot(validationFeatures, weights)

#creating the softmax activation vector for validation dataset
softmaxV = []
count = 0
for row in Zvalidation:
    count = count + 1
    data = numpy.zeros((1,10))
    for i in range(numOfClasses):
        data[0][i] = softmax_function(row[i], row)
    softmaxV.append(data)
softmaxV = numpy.array(softmaxV)

matchV = 0
for row, row1 in zip(softmaxV, validationLabels):
    t = numpy.argmax(row)
    v = numpy.argmax(row1)
    if(t == v):
        matchV = matchV + 1
accuracyV = (float(matchV)*100)/mnistValidateSize

# calculating net input for test dataset
Ztest = numpy.dot(testingFeatures, weights)

#creating the softmax activation vector for validation dataset
softmaxT = []
count = 0
for row in Ztest:
    count = count + 1
    data = numpy.zeros((1,10))
    for i in range(numOfClasses):
        data[0][i] = softmax_function(row[i], row)
    softmaxT.append(data)
softmaxT = numpy.array(softmaxT)

# calculating accuracy for test data
matchT = 0
for row, row1 in zip(softmaxT, testingLabels):
    t = numpy.argmax(row)
    v = numpy.argmax(row1)
    if(t == v):
        matchT = matchT + 1
accuracyT = (float(matchT)*100)/mnistTestSize

print("validation set accuracy = ", accuracyV)
print("test set accuracy = ", accuracyT)

validation set accuracy =  91.07
test set accuracy =  90.89


NameError: name 'classesSLR' is not defined

In [22]:
# predicted classes for test data
classesSLR = [] 
for row in softmaxT:
    classesSLR.append(numpy.argmax(row))
classesSLR=numpy.array(classesSLR)
print(classesSLR)

#confusion matrix
confusionMatrixSLR = confusion_matrix(mnistTargetTest, classesSLR)
for row in confusionMatrixSLR:
    print(row)

[7 2 1 ... 4 5 6]
[958   0   2   2   1   4   9   1   3   0]
[   0 1103    2    4    1    2    4    1   18    0]
[ 11   7 897  15  15   1  15  18  44   9]
[  5   1  20 902   1  31   4  15  21  10]
[  1   4   6   1 909   0  10   1   8  42]
[ 11   4   4  42  11 750  16  10  36   8]
[ 14   3   5   2  13  12 904   1   4   0]
[  3  19  26   5  10   0   0 927   3  35]
[  9   9   9  28   8  22  14  14 848  13]
[ 11   8   5  11  42  12   0  23   6 891]


# Training MNIST Dataset using SVM

In [8]:
# HyperParameters
kernelValue = 'rbf'
Cvalue = 2
gammaValue = 0.02

In [9]:
SVMclassifier = SVC(kernel=kernelValue, C = Cvalue, gamma = gammaValue)
SVMclassifier.fit(mnistFeaturesTrain, mnistTargetTrain)
print(SVMclassifier.score(mnistFeaturesValidate, mnistTargetValidate))
print(SVMclassifier.score(mnistFeaturesTest, mnistTargetTest))
classesSVM = SVMclassifier.predict(mnistFeaturesTest)
print(classesSVM)
confusionMatrixSVM = confusion_matrix(mnistTargetTest, classesSVM)
for row in confusionMatrixSVM:
    print(row)

0.9841
0.9835
[7 2 1 ... 4 5 6]
[974   0   1   0   0   1   1   1   2   0]
[   0 1129    2    1    0    1    0    1    1    0]
[   4    0 1014    0    1    0    1    7    4    1]
[  0   0   2 997   0   2   0   4   3   2]
[  0   0   2   0 966   0   4   0   1   9]
[  2   0   0   8   1 872   4   0   3   2]
[  4   2   0   0   2   3 946   0   1   0]
[   0    5    8    1    1    0    0 1005    1    7]
[  3   0   2   4   4   2   1   2 953   3]
[  4   2   1   6   8   2   0   6   1 979]


# Training MNIST Dataset using RandomForest

In [10]:
# HyperParameters
nEstimators = 75

In [11]:
RFclassifier = RandomForestClassifier(n_estimators = nEstimators)
RFclassifier.fit(mnistFeaturesTrain, mnistTargetTrain)
print("validation set", RFclassifier.score(mnistFeaturesValidate, mnistTargetValidate))
print("test set", RFclassifier.score(mnistFeaturesTest, mnistTargetTest))
classesRF = RFclassifier.predict(mnistFeaturesTest)
print(classesRF)
confusionMatrixRF = confusion_matrix(mnistTargetTest, classesRF)
for row in confusionMatrixRF:
    print(row)

validation set 0.971
test set 0.9684
[7 2 1 ... 4 5 6]
[971   0   1   0   0   3   1   1   3   0]
[   0 1122    2    2    0    2    3    0    3    1]
[  6   0 997   8   3   0   3   8   7   0]
[  0   0   7 975   0   8   0  10   7   3]
[  1   0   0   0 952   0   5   1   5  18]
[  3   0   0  14   3 858   4   1   6   3]
[  5   3   0   0   4   4 939   0   3   0]
[  1   3  19   1   2   0   1 991   2   8]
[  4   1   5   8   7   4   5   6 923  11]
[  6   4   1  11  13   1   0   4  13 956]


# Training MNIST Dataset using Neural Networks

#### DOWNLOADING AND PARTITIONING MNIST DATA

In [14]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
image_vector_size = 28*28
x_train = x_train.reshape(x_train.shape[0], image_vector_size)
x_test = x_test.reshape(x_test.shape[0], image_vector_size)
y_train = keras.utils.to_categorical(y_train, numOfClasses)
y_test = keras.utils.to_categorical(y_test, numOfClasses)

In [15]:
# HyperParameters
unitsLayer1 = 64
activationLayer1 = 'sigmoid'
activationLayer2 = 'softmax'
optimizerValue = 'Adadelta'
batchSize = 128
epochsValue = 20

In [16]:
image_size = 784
model = Sequential()
model.add(Dense(units=unitsLayer1, activation=activationLayer1, input_shape=(image_size,)))
model.add(Dense(units=numOfClasses, activation=activationLayer2))
model.compile(optimizer=optimizerValue, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=batchSize, epochs=epochsValue, verbose=False, validation_split=0.1)
print("validation set accuracy ", history.history['val_acc'])
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("test set loss = ", loss, "test set accuracy = " , accuracy)
classesNN = model.predict_classes(x_test, batch_size=batchSize)
print(classesNN)
confusionMatrixNN = confusion_matrix(mnistTargetTest, classesNN)
for row in confusionMatrixNN:
    print(row)

validation set accuracy  [0.907666666507721, 0.9221666668256124, 0.9298333338101705, 0.9350000004768372, 0.940833333492279, 0.9448333338101705, 0.9445, 0.9430000001589457, 0.9448333331743877, 0.9484999996821085, 0.9491666663487752, 0.9521666669845581, 0.9524999998410543, 0.9528333330154419, 0.9544999998410543, 0.950833333492279, 0.9559999996821086, 0.9558333331743876, 0.9561666669845581, 0.9544999996821085]
test set loss =  0.18010250396504998 test set accuracy =  0.9459
[7 2 1 ... 4 5 6]


# Preparing USPS Feature Dataset

In [41]:
USPSMat  = []
USPSTar  = []
curPath  = 'USPSdata/Numerals'
savedImg = []

for j in range(0,10):
    curFolderPath = curPath + '/' + str(j)
    imgs =  os.listdir(curFolderPath)
    for img in imgs:
        curImg = curFolderPath + '/' + img
        if curImg[-3:] == 'png':
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            savedImg = img
            imgdata = (255-numpy.array(img.getdata()))/255
            USPSMat.append(imgdata)
            USPSTar.append(j)

# Testing USPS Dataset on all the four classifiers

In [42]:
# SVM classifier
print("SVM USPS test data set accuracy = ", SVMclassifier.score(USPSMat, USPSTar))

# Neural Network classifier
USPSMat1 = numpy.array(USPSMat)
USPSTar1= numpy.array(USPSTar)
USPSTar1 = keras.utils.to_categorical(USPSTar1, numOfClasses)
print(USPSTar1.shape)
lossU, accuracyU = model.evaluate(USPSMat1, USPSTar1, verbose=False)
print("NN USPS test set accuracy = ", accuracyU)

# Random Forest classifier
print("RF USPS test set accuracy", RFclassifier.score(USPSMat, USPSTar))

# Softmax Logistic Regression classifier
USPSMat = numpy.insert(USPSMat, 0, 1, axis=1)
# calculating net input for test dataset
ZtestU = numpy.dot(USPSMat, weights)

#creating the softmax activation vector for validation dataset
softmaxTU = []
count = 0
for row in ZtestU:
    count = count + 1
    data = numpy.zeros((1,10))
    for i in range(numOfClasses):
        data[0][i] = softmax_function(row[i], row)
    softmaxTU.append(data)
softmaxTU = numpy.array(softmaxTU)
matchUT = 0
for row, row1 in zip(softmaxTU, USPSTar):
    t = numpy.argmax(row)
    v = numpy.argmax(row1)
    if(t == v):
        matchUT = matchUT + 1
accuracyUT = (float(matchUT)*100)/len(USPSTar)
print("SLR USPS test set accuracy", accuracyUT)

(19999, 10)
NN USPS test set accuracy =  0.21896094804740238
SLR USPS test set accuracy 9.880494024701235


# Combining Results of the 4 classifiers - Majority Voting

In [23]:
matchFinal = 0
for i in range(mnistTestSize):
    # matrix d which has 4 columns for the 4 classifiers and 10 rows for the 10 classes. Each cell in d
    # contains 1 if a classfier predicts the corresponding class for that datapoint. Every datapoint will have a d matrix.
    d = numpy.zeros((numOfClasses, 4))
    d[classesSLR[i]-1][0] = 1
    d[classesSVM[i]-1][0] = 1
    d[classesRF[i]-1][0] = 1
    d[classesNN[i]-1][0] = 1
    # matrix rankForClass contains the number of votes a particular class got by the four classifiers i.e how many classifiers 
    # voted for that particular class. So, each cell contains the sum of the corresponding rows of d. Each row in d represents
    # a class.
    rankForClass = numpy.zeros((numOfClasses))
    index = 0
    for row in d:
        rankForClass[index] = numpy.sum(row)
        index = index + 1
    # the index of rankForClass (added 1 as index starts from 0) which has the highest votes is selected as final prediction
    finalPredictedClass = numpy.argmax(rankForClass) + 1
   
    # to find the number of predictions that match with the actual predictions
    if(finalPredictedClass == mnistTargetTest[i]):
        matchFinal = matchFinal + 1

# finding accuracy
accuracyFinal = (float(matchFinal)*100)/mnistTestSize
print(accuracyFinal)

84.49
